In [179]:
import pandas as pd

In [180]:
#loading the phishing URLs data to dataframe
data_P = pd.read_csv('data/phishing_dataset.csv')
data_P.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,6557033,http://u1047531.cp.regruhosting.ru/acces-inges...,http://www.phishtank.com/phish_detail.php?phis...,2020-05-09T22:01:43+00:00,yes,2020-05-09T22:03:07+00:00,yes,Other
1,6557032,http://hoysalacreations.com/wp-content/plugins...,http://www.phishtank.com/phish_detail.php?phis...,2020-05-09T22:01:37+00:00,yes,2020-05-09T22:03:07+00:00,yes,Other
2,6557011,http://www.accsystemprblemhelp.site/checkpoint...,http://www.phishtank.com/phish_detail.php?phis...,2020-05-09T21:54:31+00:00,yes,2020-05-09T21:55:38+00:00,yes,Facebook
3,6557010,http://www.accsystemprblemhelp.site/login_atte...,http://www.phishtank.com/phish_detail.php?phis...,2020-05-09T21:53:48+00:00,yes,2020-05-09T21:54:34+00:00,yes,Facebook
4,6557009,https://firebasestorage.googleapis.com/v0/b/so...,http://www.phishtank.com/phish_detail.php?phis...,2020-05-09T21:49:27+00:00,yes,2020-05-09T21:51:24+00:00,yes,Microsoft


In [181]:
#total number of datas
data_P.shape

(14858, 8)

In [182]:
#loading the legit URLs data to dataframe
data_L = pd.read_csv('data/legit_dataset.csv')
data_L.columns = ['URLs']
data_L.head()

,URLs
0,http://1337x.to/torrent/1110018/Blackhat-2015-...
1,http://1337x.to/torrent/1122940/Blackhat-2015-...
2,http://1337x.to/torrent/1124395/Fast-and-Furio...
3,http://1337x.to/torrent/1145504/Avengers-Age-o...
4,http://1337x.to/torrent/1160078/Avengers-age-o...


In [183]:
#total number of datas
data_L.shape

(35377, 1)

# Feature Extraction
1. Search Bar Feature
2. Domain Based Feature

# 1. Search Bar Feature

In [184]:
import requests
import urllib
from urllib.request import urlopen
import whois 
import datetime 
import time
import socket
import pandas as pd 
import numpy as np
import ipaddress
import re
from urllib.parse import urlparse,urlencode
from bs4 import BeautifulSoup 

In [185]:
#This function extracts the domain from the URL by parsing it using urlparse library and then returns the domain.
def getDomain(url):
    domain = urlparse(url).netloc
    if re.match(r"^www.", domain):
        domain = domain.replace("www.", "")
    return domain

In [186]:
#This function checks if the given URL contains an IP address or not.
def havingIP(url):
  try:
    ipaddress.ip_address(url)
    ip = 1
  except:
    ip = 0
  return ip

In [187]:
#This function checks if the given URL contains an '@' symbol, which is typically not present in a standard URL.
def haveAtSign(url):
  if "@" in url:
    at = 1    
  else:
    at = 0    
  return at

In [188]:
#This function takes a URL and returns a binary value indicating whether its length is greater than or equal to 54 or not.
def getLength(url):
  if len(url) < 54:
    length = 0            
  else:
    length = 1            
  return length

In [189]:
#This function takes a URL as input and returns the depth of its path.
def getDepth(url):
  s = urlparse(url).path.split('/')
  depth = 0
  for j in range(len(s)):
    if len(s[j]) != 0:
      depth = depth+1
  return depth

In [190]:
#This function checks if there is redirection in the given URL.
def redirection(url):
  pos = url.rfind('//')
  if pos > 6:
    if pos > 7:
      return 1
    else:
      return 0
  else:
    return 0

In [191]:
#This function extracts the domain from a given URL using the urlparse method.
def httpDomain(url):
  domain = urlparse(url).netloc
  if 'https' in domain:
    return 1
  else:
    return 0

In [192]:
#listing shortening services
shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"

In [193]:
#This function checks if the given URL has been shortened using a URL shortening service by checking for the presence of the domain name of known URL shortening services in the URL.
def tinyURL(url):
    match=re.search(shortening_services,url)
    if match:
        return 1
    else:
        return 0

In [194]:
#This function checks whether the domain name in the given URL contains a hyphen or not.
def prefixSuffix(url):
    if '-' in urlparse(url).netloc:
        return 1            # phishing
    else:
        return 0            # legitimate

In [195]:
#This function takes a URL and extracts the domain name from it. 
def ippresent(url):
    domain = urlparse(url).netloc
    try:
        ip = socket.gethostbyname("domain")
        return 1
    except:
        return 0

In [196]:
#This function checks if the input URL uses HTTPS protocol or not.
def https_token(url):
    if re.findall(r"^https://", url):
        return 0
    else:
        return 1

In [197]:
#The function get_protocol_count takes a URL as input and returns the number of times the protocol (either "http" or "https") appears in the URL.
def get_protocol_count(url):
    http_count = url.count('http')
    https_count = url.count('https')
    http_count = http_count - https_count #correcting the miscount of https as http
    return (http_count + https_count)

In [198]:
#The function get_protocol takes a URL as input and returns a binary value indicating the protocol used in the URL.
def get_protocol(url):
    protocol = urlparse(url)
    if(protocol.scheme == 'http'):
        return 1
    else:
        return 0

In [199]:
#The function get_special_char_count takes a URL as input and returns the number of special characters in the URL.
def get_special_char_count(url):
    count = 0
    special_characters = [';','+=','_','?','=','&','[',']']
    for each_letter in url:
        if each_letter in special_characters:
            count = count + 1
    return count

# 2. Domain Based Feature

In [201]:
#The function dns_record takes a URL as input and returns a binary value indicating whether the domain has a DNS record or not.
def dns_record(url):
    domain_name = urlparse(url).netloc
    try:    
        rec = whois.whois(domain_name)
        return 1
    except:
        return 0      

This feature measures the popularity of the website by determining the number of visitors and the number of pages they visit. However, since phishing websites live for a short period of time, they may not be recognized by the Alexa database (Alexa the Web Information Company., 1996). By reviewing our dataset, we find that in worst scenarios, legitimate websites ranked among the top 100,000. Furthermore, if the domain has no traffic or is not recognized by the Alexa database, it is classified as “Phishing”.

In [202]:
#This function takes in a URL as an input and returns the Alexa global rank of the website.   
def web_traffic(url):
    try:
        rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find("REACH")['RANK']
        rank= int(rank)
        return rank
    except:
        return 0 #0 = Phishing

This feature can be extracted from WHOIS database. Most phishing websites live for a short period of time. The minimum age of the legitimate domain is considered to be 6 months for this project. Age here is nothing but different between creation and expiration time.

If age of domain > 6 months, the vlaue of this feature is 1 (phishing) else 0 (legitimate).

In [203]:
#The function domainAge takes a URL as input and returns the age of the domain in days.
def domainAge(url):
    try:
        domain_name = urlparse(url).netloc
    
        whois_response = whois.whois(domain_name)
        creation_date = whois_response.creation_date
        creation_date = creation_date[0].date()
        today = datetime.date.today()
        days = (today - creation_date).days
        if (days/30 < 6):
            return 1
        return 0
    except:
        return 0

In [208]:
def feature_extraction(url,label):

  feature = []

  #Search bar based features (14)
  feature.append(getDomain(url))
  feature.append(havingIP(url))
  feature.append(haveAtSign(url))
  feature.append(getLength(url))
  feature.append(getDepth(url))
  feature.append(redirection(url))
  feature.append(httpDomain(url))
  feature.append(tinyURL(url))
  feature.append(prefixSuffix(url))  
  feature.append(ippresent(url))
  feature.append(https_token(url))
  feature.append(get_protocol_count(url))
  feature.append(get_protocol(url))
  feature.append(get_special_char_count(url))

  #Domain based features (3)
  dns = 0
  try:
    domain_name = whois.whois(urlparse(url).netloc)
  except:
    dns = 1

  feature.append(dns_record(url))
  feature.append(web_traffic(url))
  feature.append(domainAge(url))

  feature.append(label)
  
  return feature

# Extracting feature to phishing URL's

In [209]:
#Collecting 50 Phishing URLs randomly
phishUrl = data_P.copy()
phishUrl = phishUrl.reset_index(drop=True)
phishUrl.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,6557033,http://u1047531.cp.regruhosting.ru/acces-inges...,http://www.phishtank.com/phish_detail.php?phis...,2020-05-09T22:01:43+00:00,yes,2020-05-09T22:03:07+00:00,yes,Other
1,6557032,http://hoysalacreations.com/wp-content/plugins...,http://www.phishtank.com/phish_detail.php?phis...,2020-05-09T22:01:37+00:00,yes,2020-05-09T22:03:07+00:00,yes,Other
2,6557011,http://www.accsystemprblemhelp.site/checkpoint...,http://www.phishtank.com/phish_detail.php?phis...,2020-05-09T21:54:31+00:00,yes,2020-05-09T21:55:38+00:00,yes,Facebook
3,6557010,http://www.accsystemprblemhelp.site/login_atte...,http://www.phishtank.com/phish_detail.php?phis...,2020-05-09T21:53:48+00:00,yes,2020-05-09T21:54:34+00:00,yes,Facebook
4,6557009,https://firebasestorage.googleapis.com/v0/b/so...,http://www.phishtank.com/phish_detail.php?phis...,2020-05-09T21:49:27+00:00,yes,2020-05-09T21:51:24+00:00,yes,Microsoft


In [210]:
data_P.shape

(14858, 8)

In [212]:
#Extracting the features and storing them in a list
phish_features = []
label = 1

for i in range(0, 14858):
  url = phishUrl['url'][i]
  phish_features.append(feature_extraction(url,label))

In [215]:
feature_names = ['Domain', 'IP', '@', 'URL Length', 'URL Depth', 'Redirection', 'https Domain', 'TinyURL', 
                'Prefix/Suffix', 'IP Present', 'HTTPS Token', 'Protocol Count', 'Protocol', 'Special Character Count', 
                'DNS Record', 'Web Traffic', 'Domain Age', 'Label']
phishing = pd.DataFrame (phish_features, columns= feature_names)
phishing.head()

,IP,@,URL Length,URL Depth,Redirection,https Domain,TinyURL,Prefix/Suffix,IP Present,HTTPS Token,Protocol Count,Protocol,Special Character Count,DNS Record,Web Traffic,Domain Age,Label
0,0,0,1,2,0,0,0,0,0,1,1,1,0,0,0,0,1
1,0,0,1,6,0,0,0,0,0,1,1,1,0,0,0,0,1
2,0,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,1
3,0,0,1,1,0,0,0,0,0,1,1,1,5,0,0,0,1
4,0,0,1,5,0,0,1,0,0,0,1,0,5,0,0,0,1


In [216]:
# Storing the extracted phishing URLs fatures to csv file
phishing.to_csv('data/phishing.csv', index= False)

# Extracting feature to legitimate URL's

In [217]:
#Collecting 50 Legitimate URLs randomly
legitUrl = data_L.copy()
legitUrl = legitUrl.reset_index(drop=True)
legitUrl.head()

,URLs
0,http://1337x.to/torrent/1110018/Blackhat-2015-...
1,http://1337x.to/torrent/1122940/Blackhat-2015-...
2,http://1337x.to/torrent/1124395/Fast-and-Furio...
3,http://1337x.to/torrent/1145504/Avengers-Age-o...
4,http://1337x.to/torrent/1160078/Avengers-age-o...


In [218]:
data_L.shape

(35377, 1)

In [219]:
#Extracting the features and storing them in a list
legit_features = []
label = 0

for i in range(0, 35377):
    url = legitUrl['URLs'][i]
    legit_features.append(feature_extraction(url,label))

In [220]:
feature_names = ['Domain', 'IP', '@', 'URL Length', 'URL Depth', 'Redirection', 'https Domain', 'TinyURL', 
                'Prefix/Suffix', 'IP Present', 'HTTPS Token', 'Protocol Count', 'Protocol', 'Special Character Count', 
                'DNS Record', 'Web Traffic', 'Domain Age', 'Label']
legitimate = pd.DataFrame (legit_features, columns= feature_names)
legitimate.head()

,IP,@,URL Length,URL Depth,Redirection,https Domain,TinyURL,Prefix/Suffix,IP Present,HTTPS Token,Protocol Count,Protocol,Special Character Count,DNS Record,Web Traffic,Domain Age,Label
0,0,0,1,3,0,0,0,0,0,1,1,1,0,0,0,0,0
1,0,0,1,3,0,0,0,0,0,1,1,1,0,0,0,0,0
2,0,0,1,3,0,0,0,0,0,1,1,1,0,0,0,0,0
3,0,0,1,3,0,0,0,0,0,1,1,1,0,0,0,0,0
4,0,0,1,3,0,0,0,0,0,1,1,1,0,0,0,0,0


In [221]:
# Storing the extracted legitimate URLs fatures to csv file
legitimate.to_csv('data/legitimate.csv', index= False)

# Final Dataset

In [222]:
#Concatenating the dataframes into one 
urldata = pd.concat([legitimate, phishing]).reset_index(drop=True)
urldata.head()

,IP,@,URL Length,URL Depth,Redirection,https Domain,TinyURL,Prefix/Suffix,IP Present,HTTPS Token,Protocol Count,Protocol,Special Character Count,DNS Record,Web Traffic,Domain Age,Label
0,0,0,1,3,0,0,0,0,0,1,1,1,0,0,0,0,0
1,0,0,1,3,0,0,0,0,0,1,1,1,0,0,0,0,0
2,0,0,1,3,0,0,0,0,0,1,1,1,0,0,0,0,0
3,0,0,1,3,0,0,0,0,0,1,1,1,0,0,0,0,0
4,0,0,1,3,0,0,0,0,0,1,1,1,0,0,0,0,0


In [223]:
urldata.tail()

,IP,@,URL Length,URL Depth,Redirection,https Domain,TinyURL,Prefix/Suffix,IP Present,HTTPS Token,Protocol Count,Protocol,Special Character Count,DNS Record,Web Traffic,Domain Age,Label
50230,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,1
50231,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,1
50232,0,0,1,3,0,0,1,0,0,1,1,1,0,0,0,0,1
50233,0,0,1,1,0,0,1,0,0,1,1,1,3,0,0,0,1
50234,0,0,1,3,0,0,1,0,0,1,1,1,0,0,0,0,1


In [224]:
urldata.shape

(50235, 17)

In [225]:
# Storing the data in CSV file
urldata.to_csv('data/combined_data.csv', index=False)